In [2]:
!pip install sqlalchemy psycopg2 pandas


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [3]:
from sqlalchemy import create_engine, MetaData
import pandas as pd
import json

# SQLite source
sqlite_engine = create_engine('sqlite:///./database.db')
sqlite_metadata = MetaData()
sqlite_metadata.reflect(bind=sqlite_engine)

user = "padel_user"
password = "portopadelleague"
host = "35.205.246.86"  # your GCE VM public IP
port = "5432"
database = "padel_league"

# PostgreSQL destination
pg_engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")

#enum_values = {
#    "prefered_hand": ["Direita", "Esquerda"],
#    "prefered_position": ["Lado direito", "Lado esquerdo", "Tanto faz"],
#    "team": ["Home", "Away"]
#}
#
#for table_name, table in sqlite_metadata.tables.items():
#    print(f"Migrating: {table_name}")
#    df = pd.read_sql_table(table_name, sqlite_engine)
#
#    # Convert dicts to JSON
#    for col in df.columns:
#        if df[col].apply(lambda x: isinstance(x, dict)).any():
#            print(f"  ➤ Converting dict column '{col}' to JSON")
#            df[col] = df[col].apply(lambda d: json.dumps(d) if isinstance(d, dict) else d)
#
#    # Convert known enums to strings
#    for col in df.columns:
#        if col in enum_values:
#            df[col] = df[col].astype(str)
#
#    df.to_sql(table_name, pg_engine, if_exists='replace', index=False)
#
#print("✅ Migration complete!")
#

In [4]:
from sqlalchemy import inspect

inspector = inspect(pg_engine)
tables = inspector.get_table_names()
print(tables)

['product_attribute_values', 'product_attributes', 'product_images', 'registrations', 'players_in_match', 'product_attribute_values_in_product', 'product_attributes_in_product', 'players_in_division', 'imageables', 'alembic_version', 'sponsor_clicks', 'divisions', 'images', 'sponsors', 'players', 'news', 'editions', 'leagues', 'matches', 'order_lines', 'orders', 'users', 'products', 'backend_app']


In [7]:
raw_conn = pg_engine.raw_connection()
try:
    df = pd.read_sql("SELECT * FROM matches", raw_conn)
finally:
    raw_conn.close()

/var/folders/9f/md89x3zj38j9zh0x6v40xsq80000gp/T/ipykernel_56773/1678563490.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM matches", raw_conn)


In [8]:
df

,id,games_home_team,games_away_team,date_hour,winner,matchweek,field,played,division_id
0,1,4.0,9.0,2022-01-12 21:30:00,-1.0,1,Campo 1,True,1
1,2,7.0,5.0,2022-01-12 21:30:00,1.0,1,Campo 1,True,1
2,3,10.0,5.0,2022-01-12 21:30:00,1.0,1,Campo 1,True,1
3,4,5.0,10.0,2022-01-12 21:30:00,-1.0,1,Campo 2,True,1
4,5,8.0,6.0,2022-01-12 21:30:00,1.0,1,Campo 2,True,1
...,...,...,...,...,...,...,...,...,...
2473,2330,5.0,7.0,2025-11-11 00:00:00,-1.0,4,Campo 1,True,57
2474,2335,7.0,8.0,2025-11-11 00:00:00,-1.0,4,Campo 1,True,57
2475,2376,7.0,5.0,2025-11-11 00:00:00,1.0,4,Campo 1,True,58
2476,2290,6.0,7.0,2025-11-11 00:00:00,-1.0,4,Campo 1,True,56


In [13]:
inspector = inspect(pg_engine)

schema_info = {}

for table_name in inspector.get_table_names():
    columns = inspector.get_columns(table_name)
    foreign_keys = inspector.get_foreign_keys(table_name)

    schema_info[table_name] = {
        "columns": [
            {
                "name": col["name"],
                "type": str(col["type"]),
                "nullable": col["nullable"],
                "default": col["default"],
            }
            for col in columns
        ],
        "foreign_keys": [
            {
                "constrained_columns": fk["constrained_columns"],
                "referred_table": fk["referred_table"],
                "referred_columns": fk["referred_columns"],
            }
            for fk in foreign_keys
        ],
    }

# --- Print / save schema description ---
print(json.dumps(schema_info, indent=2))

{
  "product_attribute_values": {
    "columns": [
      {
        "name": "id",
        "type": "BIGINT",
        "nullable": true,
        "default": "nextval('product_attribute_values_id_seq'::regclass)"
      },
      {
        "name": "value",
        "type": "TEXT",
        "nullable": true,
        "default": null
      },
      {
        "name": "product_attribute_id",
        "type": "BIGINT",
        "nullable": true,
        "default": null
      }
    ],
    "foreign_keys": []
  },
  "product_attributes": {
    "columns": [
      {
        "name": "id",
        "type": "BIGINT",
        "nullable": true,
        "default": "nextval('product_attributes_id_seq'::regclass)"
      },
      {
        "name": "name",
        "type": "TEXT",
        "nullable": true,
        "default": null
      },
      {
        "name": "user_input",
        "type": "BOOLEAN",
        "nullable": true,
        "default": null
      }
    ],
    "foreign_keys": []
  },
  "product_images": {
    

In [14]:
def format_schema_for_prompt(schema_info: dict) -> str:
    """
    Convert schema JSON into a compact text description suitable for LLM prompt context.
    """
    lines = []
    for table, info in schema_info.items():
        cols = [c["name"] for c in info["columns"]]
        col_str = ", ".join(cols)
        lines.append(f"Table {table}({col_str})")
    return "\n".join(lines)

In [15]:
formatted_schema = format_schema_for_prompt(schema_info)
print(formatted_schema)

Table product_attribute_values(id, value, product_attribute_id)
Table product_attributes(id, name, user_input)
Table product_images(id, path, product_id)
Table registrations(player_id, edition_id)
Table players_in_match(player_id, match_id, team, id)
Table product_attribute_values_in_product(product_id, product_attribute_value_id, id)
Table product_attributes_in_product(product_id, product_attribute_id, id)
Table players_in_division(player_id, division_id, place, points, appearances, percentage_of_appearances, wins, draws, losts, games_won, games_lost, matchweek, id)
Table imageables(imageable_id, type)
Table alembic_version(version_num)
Table sponsor_clicks(id, created_at, sponsor_id, ip_address, user_agent, referer, session_id, from_page)
Table divisions(id, name, beginning_datetime, rating, end_date, logo_image_path, large_picture_path, has_ended, open_division, edition_id, logo_image_id, large_picture_id)
Table images(id, filename, imageable_id, object_key, content_type, size_bytes